## Techniche - Topic Modelling

In [24]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel, atmodel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import sys

from topic_model import tokenize_docs, clean_docs, lower_words, remove_stopwords#, (TODO) Lee convert_bytes

from smart_open import smart_open

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

import pyspark
import pyspark.sql.functions as F

ImportError: cannot import name 'tokenize_docs'

In [17]:
%load_ext autoreload

# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 10
pd.set_option('display.max_rows', 10)

In [2]:
np.random.seed(3)

In [ ]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

### Import Data

#### Import data from PatentsView API

In [3]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

#### Import initial dataset

In [4]:
# build query - initial small dataset
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
# uncomment to use alternate query options
# query={"cpc_subgroup_id":"G06T3/4046"}
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
# query={"_and":
#         [{"_or":
#             [{"_text_phrase":{"patent_title":"natural language"}}
#             ,{"_text_phrase":{"patent_abstract":"natural language"}}]}
#         ,{"_and":
#       [{"patent_year":2016}]}]} 
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
response = requests.get(endpoint_url, params=params)
status = response.status_code
print("status:", status)
results = response.json()
count = results.get("count")
total_pats = results.get("total_patent_count")
print("patents on current page:",count,';', "total patents:",total_pats)

status: 200
patents on current page: 2482 ; total patents: 2482


#### Structure data

In [5]:
# extract metadata from response
print("status code:", response.status_code,';', "reason:", response.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_resp = results['patents']
# data_resp[0]

raw_df = pd.DataFrame(data_resp)
raw_df.head(3)

status code: 200 ; reason: OK
total_patent_count: 2482 ; patents_per_page: 2482


,IPCs,application_citations,applications,assignees,cited_patents,citedby_patents,cpcs,detail_desc_length,examiners,foreign_priority,...,patent_num_us_patent_citations,patent_number,patent_processing_time,patent_title,patent_type,patent_year,pct_data,rawinventors,uspcs,wipos
0,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020077823', 'ap...","[{'app_country': 'US', 'app_date': '2013-07-26...","[{'assignee_city': 'Burlington', 'assignee_cou...",[{'cited_patent_category': 'cited by examiner'...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",11570,"[{'examiner_first_name': 'Michael N', 'examine...","[{'forprior_country': None, 'forprior_date': N...",...,5,10229106,2055,Initializing a workspace for building a natura...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Jeffrey N.', 'raw...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
1,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020138265', 'ap...","[{'app_country': 'US', 'app_date': '2017-09-11...","[{'assignee_city': 'Mountain View', 'assignee_...",[{'cited_patent_category': 'cited by applicant...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",28118,"[{'examiner_first_name': 'Shreyans A', 'examin...","[{'forprior_country': None, 'forprior_date': N...",...,8,10229109,547,Allowing spelling of arbitrary words,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Evgeny A.', 'rawi...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
2,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2001/20010029455', 'ap...","[{'app_country': 'US', 'app_date': '2016-09-28...","[{'assignee_city': 'Seattle', 'assignee_countr...",[{'cited_patent_category': 'cited by applicant...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",119654,"[{'examiner_first_name': 'Jialong', 'examiner_...","[{'forprior_country': None, 'forprior_date': N...",...,26,10229113,895,Leveraging content dimensions during the trans...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Thibault Pierre',...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."


#### Subset dataframe

In [6]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id',
         'patent_firstnamed_assignee_location_id',
         'patent_firstnamed_assignee_latitude',
         'patent_firstnamed_assignee_longitude',
         'patent_firstnamed_assignee_city',
         'patent_firstnamed_assignee_state',
         'patent_firstnamed_assignee_country', 
         'patent_firstnamed_inventor_id',
         'patent_firstnamed_inventor_location_id',
         'patent_firstnamed_inventor_latitude',
         'patent_firstnamed_inventor_longitude',
         'patent_firstnamed_inventor_city',
         'patent_firstnamed_inventor_state',
         'patent_firstnamed_inventor_country',
         'patent_year', 
         'patent_type', 
         'patent_kind',
         'inventors'
            ]]
df.head(3)

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,...,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors
0,10229106,2019-03-12,Initializing a workspace for building a natura...,Designing a natural language understanding (NL...,org_ID497r4tFbCIaMBjGAST,42.5047|-71.1961,42.5047,-71.1961,Burlington,MA,...,42.3369|-71.2097,42.3369,-71.2097,Newton,MA,US,2019,utility,B2,"[{'inventor_city': 'Newton', 'inventor_country..."
1,10229109,2019-03-12,Allowing spelling of arbitrary words,"Methods, systems, and apparatus, including com...",org_p6ofWD2xFNSnyYkj6wpA,37.3861|-122.0828,37.3861,-122.083,Mountain View,CA,...,47.3119|8.5287,47.3119,8.5287,Adliswil,None,CH,2019,utility,B1,"[{'inventor_city': 'Adliswil', 'inventor_count..."
2,10229113,2019-03-12,Leveraging content dimensions during the trans...,A content management system (CMS) and a transl...,org_Vbc6obpnxWM42d0HjlXY,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,...,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,2019,utility,B1,"[{'inventor_city': 'Seattle', 'inventor_countr..."


#### Explore data

In [7]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

561

#### Create new column

In [8]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    Initializing a workspace for building a natura...
1    Allowing spelling of arbitrary words Methods, ...
2    Leveraging content dimensions during the trans...
Name: patent_title_abstract, dtype: object

In [9]:
df.sort_values(by=['patent_date'])

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,...,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors,patent_title_abstract
2481,3980994,1976-09-14,Text editing and display system having text in...,A natural language text editing and display sy...,org_1UVZxxNbuUPJkuDfvvpa,42.4906|-71.2767,42.4906,-71.2767,Bedford,MA,...,42.6583,-71.1375,Andover,MA,US,1976,utility,A,"[{'inventor_city': 'Andover', 'inventor_countr...",Text editing and display system having text in...
2480,4057849,1977-11-08,Text editing and display system,A text-editing and display system for editing ...,org_1UVZxxNbuUPJkuDfvvpa,42.4906|-71.2767,42.4906,-71.2767,Bedford,MA,...,42.6583,-71.1375,Andover,MA,US,1977,utility,A,"[{'inventor_city': 'Andover', 'inventor_countr...",Text editing and display system A text-editing...
2479,4502128,1985-02-26,Translation between natural languages,An input sentence described by a first natural...,org_70D1lR89kQnFiCFdJ6s5,35.685|139.7514,35.685,139.751,Tokyo,None,...,35.4437,139.638,Yokohama,None,JP,1985,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Translation between natural languages An input...
2478,4586160,1986-04-29,Method and apparatus for analyzing the syntact...,An automatic syntax analyzing method is applie...,org_hDziASDpeFilN1JsnVK0,35.5298|139.7024,35.5298,139.702,Kawasaki,None,...,35.4437,139.638,Yokohama,None,JP,1986,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Method and apparatus for analyzing the syntact...
2477,4599612,1986-07-08,Displaying and correcting method for machine t...,In a system wherein a first text in a first na...,org_70D1lR89kQnFiCFdJ6s5,35.685|139.7514,35.685,139.751,Tokyo,None,...,34.4783,133.933,Tama,None,JP,1986,utility,A,"[{'inventor_city': 'Fujisawa', 'inventor_count...",Displaying and correcting method for machine t...
2476,4638445,1987-01-20,Autonomous mobile robot,A vision system for a mobile robot employs at ...,None,None,None,None,None,None,...,42.1875,-71.3069,Medfield,MA,US,1987,utility,A,"[{'inventor_city': 'Medfield', 'inventor_count...",Autonomous mobile robot A vision system for a ...
2475,4661924,1987-04-28,Multiple-parts-of-speech disambiguating method...,A machine translation system comprises input m...,org_70D1lR89kQnFiCFdJ6s5,35.685|139.7514,35.685,139.751,Tokyo,None,...,35.685,139.751,Tokyo,None,JP,1987,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Multiple-parts-of-speech disambiguating method...
2474,4688195,1987-08-18,Natural-language interface generating system,A system for interactively generating a natura...,org_C6OV0HJ5yTbw8A4Wv6Ag,32.7833|-96.8,32.7833,-96.8,Dallas,TX,...,33.0197,-96.6986,Plano,TX,US,1987,utility,A,"[{'inventor_city': 'Plano', 'inventor_country'...",Natural-language interface generating system A...
2473,4689737,1987-08-25,Integrated environment computer system control...,A computer system including text input and dis...,None,None,None,None,None,None,...,34.1511,-118.448,Sherman Oaks,CA,US,1987,utility,A,"[{'inventor_city': 'Sherman Oaks', 'inventor_c...",Integrated environment computer system control...
2472,4695975,1987-09-22,Multi-image communications system,A device for automatic translation of natural ...,org_rjvvHMFnrkq3zHsiW0ej,40.7142|-74.0064,40.7142,-74.0064,New York,NY,...,41.7003,-73.9214,Poughkeepsie,NY,US,1987,utility,A,"[{'inventor_city': 'Poughkeepsie', 'inventor_c...",Multi-image communications system A device for...


In [10]:
text_data = df.patent_title_abstract.tolist()
text_data[:3]

['Initializing a workspace for building a natural language understanding system Designing a natural language understanding (NLU) model for an application from scratch can be difficult for non-experts. A system can simplify the design process by providing an interface allowing a designer to input example usage sentences and build an NLU model based on presented matches to those example sentences. In one embodiment, a method for initializing a workspace for building an NLU system includes parsing a sample sentence to select at least one candidate stub grammar from among multiple candidate stub grammars. The method can include presenting, to a user, respective representations of the candidate stub grammars selected by the parsing of the sample sentence. The method can include enabling the user to choose one of the respective representations of the candidate stub grammars. The method can include adding to the workspace a stub grammar corresponding to the representation of the candidate stu

In [11]:
# partition data
len(text_data)
text_train = text_data[:round(len(text_data)*.8)]
text_test = text_data[round(len(text_data)*.8):]
print(len(text_data), len(text_train), len(text_test), len(text_train)+len(text_test))

2482 1986 496 2482


### Pre-process text data

In [ ]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

#### Tokenize

In [12]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(text_train)

#### Clean punctuation

In [13]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [ ]:
cleaned_data = clean_docs(tokenized_docs)
cleaned_data[0]

#### Convert to lowercase

In [14]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lowered_data = lower_words(cleaned_data)
lowered_data[0]

NameError: name 'cleaned_data' is not defined

#### Clean stopwords

In [ ]:
# clean stopwords

stop_words = stopwords.words('english')

In [ ]:
def filter_stopwords(docs):
    filtered_docs = []
    for doc in docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
filtered_data = filter_stopwords(lowered_data)
filtered_data
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

#### Construct bigrams and trigrams

In [ ]:
# train bigram phrases model
bigram_model = Phrases(filtered_data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[filtered_data], threshold=100)  

In [ ]:
# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

In [ ]:
# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

In [ ]:
bigrams(filtered_data)[0]

In [ ]:
# def trigrams(docs):
#     """create trigrams"""
#     return [trigram_model[bigram_model[doc]] for doc in docs]

In [ ]:
# trigrams(filtered_data)[0]

#### Stem and Lemmatize

In [ ]:
# def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """lemmatize documents"""
#     lemmatized_docs = []
#     for doc in docs: 
#         lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return lemmatized_docs

In [ ]:
# # TODO (Lee)
# # TODO (Lee) - lemmatize_docs(cleaned_data)

# lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
# # for doc in cleaned_data:
# #     for token in doc:
# #         token.lemma_

# # uncomment to use
# # download english model with "python -m spacy download en"

# # for token in doc:
# #     print(token, token.lemma, token.lemma_)



In [ ]:
len(texts)

#### Create dictionary and convert tokens into frequency counts by document

In [ ]:
# specify corpus - list of patent-list of tokenized words
texts = filtered_data

In [ ]:
# build dictionary – a mapping between words and their integer ids
id_to_word = corpora.Dictionary(filtered_data)

In [ ]:
# .dfs returns frequency of documents containing given token in tuple (token_id, count of documents that contain this token)
id_to_word.dfs.items()

In [ ]:
# apply term document frequency - convert docs in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(text) for text in texts]
corpus

In [ ]:
corpus[0]

In [ ]:
# view formatted corpus (term-doc-frequency)
[[(id_to_word[id], freq) for id, freq in text] for text in corpus]

### Model - model #1

In [ ]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

In [ ]:
doc = corpus[0]

In [ ]:
def get_topics(doc, k=3, model_lda=model_lda):
    topic_id = sorted(model_lda[doc][0], key=lambda x: -x[1])
    top_k_topics = [x[0] for x in topic_id[:k]]
    return [(i, model_lda.print_topic(i)) for i in top_k_topics]

In [ ]:
topic_id

In [ ]:
doc_id = 675
get_topics(corpus[doc_id], k=5), text_train[doc_id]

In [ ]:
# text = 'virtual dictionary lexicon enablement voice'.split()
text = 'smart assistant transformer model translation'.split()

In [ ]:
def keywords_string(keywords):
    keyword_string = str(keywords)
    return keyword_string

In [22]:
def process_keywords(keywords):
    from topic_model import tokenize_docs, clean_docs, lower_words, remove_stopwords
    tokenized_text = str(keywords).split()
    cleaned_text = clean_docs(tokenized_docs)
    lowered_text = lower_words(cleaned_data)
    return lowered_text

In [23]:
text = process_keywords("smart assistant transformer model translation")

ImportError: cannot import name 'tokenize_docs'

In [ ]:
text

In [ ]:
get_topics(id_to_word.doc2bow(text))

In [ ]:
def get_documents()

In [ ]:
text_train[0]

In [ ]:
model_lda[doc]

In [ ]:
# print keywords in n topics
sorted(model_lda.print_topics(), key=lambda x: x[0])

In [ ]:
# print keywords in n topics
pprint(model_lda.print_topics())

In [ ]:
# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

In [ ]:
# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

In [ ]:
# TODO (Lee) - infer topic from keywords?

### Evaluate - model #1

In [ ]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

In [ ]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_data, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

In [ ]:
model_lda.

In [ ]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

In [ ]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_to_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

### Model 2-  Mallet model

In [ ]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [ ]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_to_word)

In [ ]:
# topics
pprint(model_2.show_topics(formatted=False))

In [ ]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=filtered_data, dictionary=id_to_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

### Model 3 - Author topic model

#### pre-process

In [ ]:
tokenized_docs_at = tokenize_docs(text_data)
cleaned_data_at = clean_docs(tokenized_docs_at)
lowered_data_at = lower_words(cleaned_data_at)
filtered_data_at = filter_stopwords(lowered_data_at)

In [ ]:
len(filtered_data_at)

#### build dictionary and corpus from processed text

In [ ]:
# build dictionary
id_to_word_at = corpora.Dictionary(filtered_data_at)

# build corpus
texts_at = filtered_data_at

# apply term document frequency - converts docs in corpus to bag-of-words format via list of (token_id, token_count) tuples
corpus_at = [id_to_word_at.doc2bow(text) for text in texts_at]

In [ ]:
(next(iter(id_to_word_at.items())))

In [ ]:
type(id_to_word_at.keys()[0])

In [ ]:
type(id_to_word_at.values())

#### construct inventor-doc mapping from nested inventors column in json api response

In [ ]:
# extract nested inventors table from api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.pop("patent_date")
df_inventors.head(3)

In [ ]:
# TODO (Lee) - resolve workaround
# df_idx = df
# df_idx['idx'] = df.index
# df_idx
# # df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
# df_idx_2 = df_idx_1.set_index('patent_number')
# df_idx_2.pop('inventors')
# df_idx_2
# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

In [ ]:
dict_pat2inv =df_inventors.set_index('patent_number').T.to_dict('list')
# dict_pat2inv

In [ ]:
# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

# idx_pat_map = df.patent_number.to_dict()
# idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
# import itertools
# x = list(itertools.islice(idx_pat_map.items(), 0, 4))
# x[:4]

In [ ]:
pat2inv_dict = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}

In [ ]:
# {k: list(v) for k,v in df_pat2inv.groupby("patent_number")["inventor_id"]}

In [ ]:
# df2 = df_inventors.groupby("patent_number")["inventor_id"]

In [ ]:
# df3 = df_idx_pat_inv_map.groupby("patent_number")["inventor_id"]

In [ ]:
pat2inv = {k: list(str(v)) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
len(pat2inv.items())

In [ ]:
pat2inv.items()
type(next(iter(pat2inv)))

In [ ]:
pat2inv_2 = {str(k): list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
len(pat2inv_2)

In [ ]:
patdf2inv_2 = dict((df_inventors[key], value) for (key, value) in pat2inv.items())
patdf2inv_2

In [ ]:
patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
patdf2inv

#### Construct author-topic model

In [ ]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus_at,
                         doc2author=patdf2inv,
                         id2word=id_to_word_at, 
                         num_topics=25)

In [ ]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

In [ ]:
# inspect topic distribution for author with id# 7788103-1
# each topic has a probability of being expressed given the particular inventor, but only the ones above a certain threshold are shown.

model_at['7788103-1']

In [ ]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

In [ ]:
gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)